In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage.metrics as sk
from sewar.full_ref import vifp
from skimage import io, filters

### FR-IQMs

In [2]:
# Mean Squared Error(MSE)
def MSE(img_ref_color, img_color):

    mse_color = np.zeros(3)

    for i in range(3):
        mse_channel = sk.mean_squared_error(img_ref_color[:,:,i], img_color[:,:,i])
        mse_color[i] = (mse_channel)
    
    mse = np.mean((mse_color))

    return mse
# Peak Signal to Noise Ratio(PSNR)
def PSNR(img_ref_color, img_color):

    PSNR_color = np.zeros(3)

    for i in range(3):
        PSNR_channel = sk.peak_signal_noise_ratio(img_ref_color[:,:,i], img_color[:,:,i])
        PSNR_color[i] = (PSNR_channel)
    
    PSNR_img = np.mean((PSNR_color))

    return PSNR_img

# Signal to Noise Ratio(SNR)
def SNR(img_ref_color, img_color):

    SNR_color = np.zeros(3)
    for i in range(3):
        power_img = np.mean(np.square(img_ref_color[:,:,i]))

        MSE = sk.mean_squared_error(img_ref_color[:,:,i], img_color[:,:,i])

        SNR = 10*np.log10(power_img/MSE)

        SNR_color[i] = (SNR)
    
    SNR_img = np.mean((SNR_color))
        
    return SNR_img
# Structural Content(SC)
def SC(img_ref_color, img_color):

    SC_color = np.zeros(3)

    for i in range(3):

        power_ref = np.sum(np.square(img_ref_color[:,:,i]))
        power_img = np.sum(np.square(img_color[:,:,i]))

        SC_color[i] = (power_ref/power_img)

    SC_img = np.mean((SC_color))

    return SC_img

# Maximum Difference(MD)
def MD(img_ref_color, img_color):

    MD_color = np.zeros(3)

    for i in range(3):
        temp = np.absolute(img_ref_color[:,:,i] - img_color[:,:,i])
        MD_color[i] = (np.max(temp))

    MD_img = np.mean((MD_color))
        
    return MD_img

# Average Difference(AD)
def AD(img_ref_color, img_color):

    AD_color = np.zeros(3)
    for i in range(3):
        AD_color[i] = (np.mean(img_ref_color[:,:,i] - img_color[:,:,i]))
    
    AD_img = np.mean((AD_color))
    
    return AD_img

# Normalized Absolute Error(NAE)
def NAE(img_ref_color, img_color):

    NAE_color = np.zeros(3)
    for i in range(3):

        num = np.sum(np.absolute(img_ref_color[:,:,i] - img_color[:,:,i]))
        den = np.sum(np.absolute(img_ref_color[:,:,i]))

        NAE_color[i] = (num/den)
    NAE_img = np.mean((NAE_color))

    return NAE_img

# R-Averaged Maximum Difference(RAMD) 
def RAMD(img_ref_color, img_color, R):

    RAMD_color = np.zeros(3)
    for i in range(3):
        arr = np.sort(np.absolute(img_ref_color[:,:,i] - img_color[:,:,i]), axis=None)
        arr1 = arr[-1:-R-1:-1]
        RAMD_color[i] = (np.mean(arr1))

    RAMD_img = np.mean((RAMD_color))
    return RAMD_img

# Laplacian Mean Squared Error(LMSE)
def LMSE(img_ref_color, img_color):

    LMSE_color = np.zeros(3)

    for k in range(3):

        img_ref = img_ref_color[:,:,k]
        img = img_color[:,:,k]

        img_ref_pad = np.zeros((img_ref.shape[0]+1,img_ref.shape[1]), dtype=np.int_)
        img_pad = np.zeros((img.shape[0]+1,img.shape[1]), dtype=np.int_)

        img_ref_pad[1:,:] = img_ref
        img_ref_pad[0,:] = img_ref[0,:]
        
        img_pad[1:,:] = img
        img_pad[0,:] = img[0,:]

        h_img = np.zeros((img.shape[0]-1, img.shape[1]-2), dtype=np.int_)
        h_img_ref = np.zeros((img_ref.shape[0]-1, img_ref.shape[1]-2), dtype=np.int_)

        for i in range(img.shape[0]-1):
            for j in range(img.shape[1]-2):

                h_img[i][j] = img_pad[i+2][j+1] + img_pad[i][j+1] + img_pad[i+1][j] + img_pad[i+1][j+2] - 4*img_pad[i+1][j+1]
                h_img_ref[i][j] = img_ref_pad[i+2][j+1] + img_ref_pad[i][j+1] + img_ref_pad[i+1][j] + img_ref_pad[i+1][j+2] - 4*img_ref_pad[i+1][j+1]
        
        num = np.sum(np.square(h_img_ref - h_img))
        den = np.sum(np.square(h_img_ref))
    
        LMSE_color[k] = (num/den)

    LMSE_img = np.mean((LMSE_color))

    return LMSE_img

# Normalised Cross-Correlation(NXC)
def NXC(img_ref_color, img_color):
    NXC_color = np.zeros(3)
    for i in range(3):

        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        num = np.sum(np.multiply(img_ref, img))
        den = np.sum(np.square(img_ref))
        NXC_color[i]=(num/den)

    NXC_img = np.mean(NXC_color)

    return NXC_img

# Mean Angle Similarity(MAS)
def MAS(img_ref, img):

    I = img
    I_cap = img_ref
    
    height = I.shape[0]
    width = I.shape[1]
    s = 0

    for i in range(height):
        for j in range(width):
            
            x   = np.linalg.norm(I[i,j])
            y = np.linalg.norm(I_cap[i,j])

            if (x!=0) and (y!=0):
                alpha = (2*np.dot(I[i,j],I_cap[i,j]))/(np.pi*np.linalg.norm(I[i,j])*np.linalg.norm(I_cap[i,j]))
            
            else: 
                alpha = 0
            s = s + alpha
    
    MAS = 1 - (s/(height*width))

    return MAS

# Mean Angle Magnitude Similarity(MAMS) 
def MAMS(img_ref, img):

    I = img
    I_cap = img_ref
    
    height = I.shape[0]
    width = I.shape[1]
    s = 0

    for i in range(height):
        for j in range(width):
            x   = np.linalg.norm(I[i,j])
            y = np.linalg.norm(I_cap[i,j])

            if (x!=0) and (y!=0):
                alpha = (2*np.dot(I[i,j],I_cap[i,j]))/(np.pi*np.linalg.norm(I[i,j])*np.linalg.norm(I_cap[i,j]))
            
            else: 
                alpha = 0
            mag =1 - ((np.linalg.norm(I[i,j]-I_cap[i,j]))/(np.sqrt(3)*255))
            s = s + (1 - ((1-alpha)*mag))
    
    MAMS = (s/(height*width))

    return MAMS

# Total Edge Difference(TED)
def TED(img_ref_color, img_color):

    TED_color = np.zeros(3)

    for i in range(3):
    
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        grad_map_ref = filters.sobel(img_ref)
        grad_map = filters.sobel(img)

        TED_val = np.sum(np.abs(grad_map-grad_map_ref))

        TED_color[i]= (TED_val)

    TED_img = np.mean(TED_color)

    return TED_img

# Total Corner Difference(TCD)
def TCD(img_ref_color, img_color):

    TCD_color = np.zeros(3)

    for i in range(3):

        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        img_ref_32 = np.float32(img_ref)
        img_32 = np.float32(img)
        
        dest_ref = cv2.cornerHarris(img_ref_32, 2, 3, 0.04)
        dest = cv2.cornerHarris(img_32, 2, 3, 0.04)
        
        threshold_ref = 0.01 * dest_ref.max()
        threshold = 0.01 * dest.max()

        corner_ref_img = np.zeros_like(dest_ref)
        corner_img = np.zeros_like(dest)

        corner_ref_img[dest_ref > threshold_ref] = 255
        corner_img[dest > threshold] = 255

        N_cr = np.count_nonzero(corner_ref_img)
        N_cap_cr = np.count_nonzero(corner_img)

        TCD_val = np.absolute(N_cr - N_cap_cr)/max(N_cr, N_cap_cr)

        TCD_color[i]=TCD_val

    TCD_img = np.mean(TCD_color)

    return TCD_img

# Spectral Magnitude Error(SME)
def SME(img_ref_color, img_color):

    SME_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]

        F_ref = cv2.dft(np.float32(img_ref),flags = cv2.DFT_COMPLEX_OUTPUT)
        F = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)

        mag_F_ref = cv2.magnitude(F_ref[:,:,0], F_ref[:,:,1])
        mag_F = cv2.magnitude(F[:,:,0], F[:,:,1])

        SME_color[i] = np.mean(np.square(mag_F_ref - mag_F))
    
    SME_img = np.mean(SME_color)

    return SME_img

# Spectral Phase Error(SPE)
def SPE(img_ref_color, img_color):

    SPE_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        
        F_ref = cv2.dft(np.float32(img_ref),flags = cv2.DFT_COMPLEX_OUTPUT)
        F = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)

        phase_ref = cv2.phase(F_ref[:,:,0], F_ref[:,:,1], angleInDegrees=False)
        phase = cv2.phase(F[:,:,0], F[:,:,1], angleInDegrees=False)

        temp = np.square(np.absolute(phase_ref - phase))

        SPE_color[i] = np.mean(temp)

    SPE_img = np.mean(SPE_color)

    return SPE_img

# Gradient Magnitude Error(GME)
def GME(img_ref_color, img_color):

    GME_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]

        G_ref_x = cv2.Sobel(img_ref, cv2.CV_64F, 1, 0)
        G_ref_y = cv2.Sobel(img_ref, cv2.CV_64F, 0, 1)
        G_ref = np.vectorize(complex)(G_ref_x, G_ref_y)

        G_x = cv2.Sobel(img, cv2.CV_64F, 1, 0)
        G_y = cv2.Sobel(img, cv2.CV_64F, 0, 1)
        G = np.vectorize(complex)(G_x, G_y)

        temp = np.absolute(G_ref) - np.absolute(G)

        GME_color[i] = np.mean(np.square(temp))

    GME_img = np.mean(GME_color)

    return GME_img
# Gradient Phase Error(GPE)
def GPE(img_ref_color, img_color):

    GPE_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        
        img_ref = cv2.cvtColor(img_ref_color, cv2.COLOR_BGR2GRAY)
        img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        G_ref_x = cv2.Sobel(img_ref, cv2.CV_64F, 1, 0)
        G_ref_y = cv2.Sobel(img_ref, cv2.CV_64F, 0, 1)
        G_ref = np.vectorize(complex)(G_ref_x, G_ref_y)

        G_x = cv2.Sobel(img, cv2.CV_64F, 1, 0)
        G_y = cv2.Sobel(img, cv2.CV_64F, 0, 1)
        G = np.vectorize(complex)(G_x, G_y)
        
        temp = np.absolute(np.angle(G_ref) - np.angle(G))
        GPE_color[i] = np.mean(np.square(temp))

    GPE_img = np.mean(GPE_color)
    return GPE_img

# Structural Similarity Index(SSIM)
def SSIM(img_ref_color, img_color):

    SSIM_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]      
        SSIM_color[i] =  sk.structural_similarity(img_ref, img)

    SSIM_img = np.mean(SSIM_color)

   
    return SSIM_img

# Visual Information Fidelity(VIF)
def VIF(img_ref_color, img_color):

    VIF_color = np.zeros(3)

    for i in range(3):
        img_ref = img_ref_color[:,:,i]
        img = img_color[:,:,i]
        VIF_color[i] = vifp(img_ref, img)

    VIF_img = np.mean(VIF_color)
    return VIF_img

##### NR-IQMs

In [3]:
# JPEG Quality Index

def measures_JPEG_INDEX(img):

    height = int(img.shape[0])
    width = int(img.shape[1])

    difference_image = np.zeros((height,width-1))

    for i in range(height):

        for j in range(width-1):

            difference_image[i,j] = img[i,j+1] - img[i,j]

    B = 0

    for i in range(height):

        for j in range(0, (int(np.floor(width/8))-1) ):

            B = B + np.abs(difference_image[i,8*j])

    scaling = 1/(height*((np.floor(width/8))-1))

    B_final = scaling*B

    A = 0

    for i in range(height):

        for j in range(0, width-1):

            A = A + (np.abs(difference_image[i,j]) - B_final)

    scaling = 8/(7*height*(width-1))

    A_final = scaling*A

    zero_cross = np.zeros((height,width-2))

    for i in range(height):

        for j in range(0, width-2):

            if difference_image[i,j]*difference_image[i,j+1] < 0 :
            
                zero_cross[i,j] = 1

            elif j+2 <= width-2 :

                if (difference_image[i,j]*difference_image[i,j+1] == 0) and (difference_image[i,j]*difference_image[i,j+2]< 0):
            
                    zero_cross[i,j] = 1

    Z_final = np.average(zero_cross)

    return (B_final,A_final,Z_final)

def JPEG_quality_index(img1):

    img_color = np.array(img1).astype(int)

    JPEG_index_color = np.zeros(3)

    for i in range(3):

        img = img_color[:,:,i]

        Bh,Ah,Zh = measures_JPEG_INDEX(img)

        Bv,Av,Zv = measures_JPEG_INDEX(np.transpose(img))

        alpha = -245.9

        beta = 261.9

        gamma1 = -0.0240

        gamma2 = 0.0160

        gamma3 = 0.0064

        B = 0.5*(Bh + Bv)
        A = 0.5*(Ah + Av)
        Z = 0.5*(Zh + Zv)

        quality_index = alpha + beta*( (B*(gamma1)) + (A*(gamma2)) + (Z*(gamma3)))

        JPEG_index_color[i] = quality_index

    JPEG_index_img = np.mean(JPEG_index_color)

    return JPEG_index_img


# High-Low Frequency Index
def HLFI(img_color):

    HLFI_color = np.zeros(3)

    for i in range(3):
        img = img_color[:,:,i]      
        height = img.shape[0]
        width = img.shape[1]

        i_l = (15*height)//100
        j_l = (15*width)//100

        F = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
        mag_F = cv2.magnitude(F[:,:,0], F[:,:,1])
        mag_F1 = mag_F[:i_l, :j_l]
        mag_F2 = mag_F[i_l:, j_l:]

        num = np.sum(mag_F1) - np.sum(mag_F2)
        den = np.sum(mag_F)

        HLFI_color[i] = num/den

    HLFI_img = np.mean(HLFI_color)

    return HLFI_img

# Blind Image Quality Index(BIQI)
def BIQI(img):
    pass
       

### Feature Vector

In [4]:
def feature_vector(img):

    img_ref = cv2.GaussianBlur(img, (3, 3), 0.5)
    vector = []

    vector.append(MSE(img, img_ref))
    vector.append(PSNR(img, img_ref))
    vector.append(SNR(img, img_ref))
    vector.append(SC(img, img_ref))
    vector.append(MD(img, img_ref))
    vector.append(AD(img, img_ref))
    vector.append(NAE(img, img_ref))
    vector.append(RAMD(img, img_ref, 10))
    vector.append(LMSE(img, img_ref))
    vector.append(NXC(img, img_ref))
    vector.append(MAS(img, img_ref))
    vector.append(MAMS(img, img_ref))
    vector.append(TED(img, img_ref))
    vector.append(TCD(img, img_ref))
    vector.append(SME(img, img_ref))
    vector.append(SPE(img, img_ref))
    vector.append(GME(img, img_ref))
    vector.append(GPE(img, img_ref))
    vector.append(SSIM(img, img_ref))
    vector.append(VIF(img, img_ref))
    vector.append(JPEG_quality_index(img))
    vector.append(HLFI(img))

    return np.array(vector)

#### Input Dataset

REPLAY ATTACK

In [5]:
def load_images_from_folder(folder,l):

    images = []
    labels = []
    size = 64
    for label in os.listdir(folder):
        
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):

            for filename in os.listdir(label_path):

                img_path = os.path.join(label_path, filename)
                if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    
                    img = cv2.resize(cv2.imread(img_path),(size,size))
                    if img is not None:

                        images.append(img)
                        labels.append(l)
                    

    return images, labels

# Train Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\real"
replay_img_real_train, replay_label_r_train = load_images_from_folder(path, 1)

# Train Dataset attack
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\attack\hand"
replay_img_spoof_train_hand, replay_label_s_train_hand = load_images_from_folder(path, -1)

path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\attack\fixed"
replay_img_spoof_train_fixed, replay_label_s_train_fixed = load_images_from_folder(path, -1)

replay_img_spoof_train = replay_img_spoof_train_fixed + replay_img_spoof_train_hand 
replay_label_s_train = replay_label_s_train_fixed + replay_label_s_train_hand

replay_img_train = np.array(replay_img_real_train + replay_img_spoof_train)
replay_label_train = np.array(replay_label_r_train + replay_label_s_train)

# test Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\real"
replay_img_real_test, replay_label_r_test = load_images_from_folder(path, 1)

# test Dataset attack
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\attack\hand"
replay_img_spoof_test_hand, replay_label_s_test_hand = load_images_from_folder(path, -1)

path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\attack\fixed"
replay_img_spoof_test_fixed, replay_label_s_test_fixed = load_images_from_folder(path, -1)

replay_img_spoof_test = replay_img_spoof_test_fixed + replay_img_spoof_test_hand 
replay_label_s_test = replay_label_s_test_fixed + replay_label_s_test_hand

replay_img_test = np.array(replay_img_real_test + replay_img_spoof_test)
replay_label_test = np.array(replay_label_r_test + replay_label_s_test)

LCC

In [6]:
def load_images_from_folder_modified(folder,l):

    images = []
    labels = []
    size = 64
    for label in os.listdir(folder):
        #for filename in os.listdir(label):

        img_path = os.path.join(folder, label) 
        #os.path.join(label, filename)
        
        if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img = cv2.resize(cv2.imread(img_path),(size,size))
            if img is not None:

                images.append(img)
                labels.append(l)
                    
    return images, labels

# Train Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_training\real"
LCC_img_real_train, LCC_label_r_train = load_images_from_folder_modified(path, 1)

# Train Dataset attack
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_training\spoof"
LCC_img_spoof_train, LCC_label_s_train = load_images_from_folder_modified(path, -1)

LCC_img_train = np.array(LCC_img_real_train + LCC_img_spoof_train)
LCC_label_train = np.array(LCC_label_r_train + LCC_label_s_train)

# test Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_evaluation\real"
LCC_img_real_test, LCC_label_r_test = load_images_from_folder_modified(path, 1)

# test Dataset attack
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_evaluation\spoof"
LCC_img_spoof_test, LCC_label_s_test = load_images_from_folder_modified(path, -1)

LCC_img_test = np.array(LCC_img_real_test + LCC_img_spoof_test)
LCC_label_test = np.array(LCC_label_r_test + LCC_label_s_test)


NUAA

In [7]:
def load_images_from_folder(folder,l):

    images = []
    labels = []
    size = 64

    for label in os.listdir(folder):
        
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):

            for filename in os.listdir(label_path):

                img_path = os.path.join(label_path, filename)
                if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):

                    img = cv2.resize(cv2.imread(img_path),(size,size))
                    if img is not None:

                        images.append(img)
                        labels.append(l)

    return images, labels

# Original Faces(Client) with label 1
path = r"F:\IVP_Project\Dataset\Dataset\nuaa_cropped\NUAA\Detectedface\ClientFace"
img_real, label_r = load_images_from_folder(path, 1)

# Spoof Faces with label -1
path = r"F:\IVP_Project\Dataset\Dataset\nuaa_cropped\NUAA\Detectedface\ImposterFace"
img_spoof, label_s = load_images_from_folder(path, -1)

#### Feature Vector Generation

Replay Attack

In [8]:
from tqdm import tqdm

# FOR REPLAY ATTACK DATASET
feature_vec_replay_train = []
feature_vec_replay_test = []

replay_train_images = replay_img_train
replay_test_images = replay_img_test

# Loop over images in image_real with tqdm progress bar
for img in tqdm(replay_train_images, desc="Processing replay dataset train images"):
    feature_vec_replay_train.append(feature_vector(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(replay_test_images, desc="Processing replay dataset test images"):
    feature_vec_replay_test.append(feature_vector(img))

# Convert lists to numpy arrays
feature_vector_replay_train = np.array(feature_vec_replay_train)
feature_vector_replay_test = np.array(feature_vec_replay_test)

nique_replay_train = np.genfromtxt(r'niqe_train_replay.csv', delimiter=',')
nique_replay_test = np.genfromtxt(r'niqe_test_replay.csv', delimiter=',')

nique_replay_train = nique_replay_train.reshape((1, nique_replay_train.shape[0]))
feature_vector_replay_train = np.concatenate((feature_vector_replay_train, nique_replay_train.T), axis=1)

nique_replay_test = nique_replay_test.reshape((1, nique_replay_test.shape[0]))
feature_vector_replay_test = np.concatenate((feature_vector_replay_test, nique_replay_test.T), axis=1)

Processing replay dataset test images: 100%|██████████| 4800/4800 [21:25<00:00,  3.73it/s]


LCC

In [9]:
from tqdm import tqdm

# FOR LCC DATASET
feature_vec_LCC_train = []
feature_vec_LCC_test = []

LCC_train_images = LCC_img_train
LCC_test_images = LCC_img_test

# Loop over images in image_real with tqdm progress bar
for img in tqdm(LCC_train_images, desc="Processing LCC dataset train images"):
    feature_vec_LCC_train.append(feature_vector(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(LCC_test_images, desc="Processing LCC dataset test images"):
    feature_vec_LCC_test.append(feature_vector(img))

# Convert lists to numpy arrays
feature_vector_LCC_train = np.array(feature_vec_LCC_train)
feature_vector_LCC_test = np.array(feature_vec_LCC_test)

nique_LCC_train = np.genfromtxt(r'niqe_train_lcc.csv', delimiter=',')
nique_LCC_test = np.genfromtxt(r'niqe_test_lcc.csv', delimiter=',')

nique_LCC_train = nique_LCC_train.reshape((1, nique_LCC_train.shape[0]))
feature_vector_LCC_train = np.concatenate((feature_vector_LCC_train, nique_LCC_train.T), axis=1)

nique_LCC_test = nique_LCC_test.reshape((1, nique_LCC_test.shape[0]))
feature_vector_LCC_test = np.concatenate((feature_vector_LCC_test, nique_LCC_test.T), axis=1)

Processing LCC dataset test images: 100%|██████████| 7635/7635 [33:20<00:00,  3.82it/s]


NUAA

In [10]:
from tqdm import tqdm

feature_vec_real = []
feature_vec_spoof = []

image_real = img_real
image_spoof = img_spoof

# Loop over images in image_real with tqdm progress bar
for img in tqdm(image_real, desc="Processing real images"):
    feature_vec_real.append(feature_vector(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(image_spoof, desc="Processing spoof images"):
    feature_vec_spoof.append(feature_vector(img))

# Convert lists to numpy arrays
feature_vectors_real = np.array(feature_vec_real)
feature_vectors_spoof = np.array(feature_vec_spoof)

niqe_real = np.genfromtxt(r'niqe_client.csv', delimiter=',')

niqe_real = niqe_real.reshape((1, niqe_real.shape[0]))
feature_vectors_real = np.concatenate((feature_vectors_real, niqe_real.T), axis=1)

niqe_imposter = np.genfromtxt(r'niqe_imposter.csv', delimiter=',')
niqe_imposter = niqe_imposter.reshape((1, niqe_imposter.shape[0]))
feature_vectors_spoof = np.concatenate((feature_vectors_spoof, niqe_imposter.T), axis=1)


Processing spoof images: 100%|██████████| 7509/7509 [32:02<00:00,  3.91it/s]


In [11]:
from sklearn.model_selection import train_test_split

# Concatenating the Dataset

data_set = np.concatenate((feature_vectors_real, feature_vectors_spoof), axis=0)
data_set_label = np.concatenate((label_r, label_s),axis=0)

# Shuffling teh dataset
shuffle_index = np.random.permutation(len(data_set_label))

dataset, d_labels = data_set[shuffle_index,:], data_set_label[shuffle_index]

# Split the data into training and testing sets
nuaa_train, nuaa_test, nuaa_label_train, nuaa_label_test = train_test_split(dataset, d_labels, test_size=0.2, random_state=42)

In [12]:
# Arranging the dataset

NUAA = {"train_data":nuaa_train, "test_data":nuaa_test, "train_label":nuaa_label_train, "test_label":nuaa_label_test}
LCC = {"train_data":feature_vector_LCC_train,"test_data":feature_vector_LCC_test,"train_label":LCC_label_train,"test_label":LCC_label_test}
REPLAY = {"train_data":feature_vector_replay_train,"test_data":feature_vector_replay_test,"train_label":replay_label_train,"test_label":replay_label_test}
data_set = {"NUAA":NUAA,"LCC":LCC,"REPLAY":REPLAY}

In [13]:
# Save each numpy array in each dictionary as .npy files
for dataset_name, dataset_content in data_set.items():
    for data_name, data_array in dataset_content.items():
        np.save(f"{dataset_name}_{data_name}.npy", data_array)